# Notebook 8 (Industrial Edition): Decentralized Blackboard Collaboration

## Introduction: Enabling Emergent Intelligence

This notebook explores the **Decentralized Blackboard Collaboration** pattern, a highly flexible and powerful architecture for multi-agent systems. Unlike rigid pipelines or hierarchies, a blackboard system consists of a shared data space (the blackboard) and a collection of independent, specialist agents who watch it. Agents activate themselves opportunistically when the state of the blackboard matches their expertise, contributing their knowledge and incrementally building towards a solution.

### The Core Concept: Decoupled, Event-Driven Collaboration

The blackboard holds the current state of the problem. Specialist agents don't communicate directly with each other. They only read from and write to the blackboard. A central controller (in our case, a router in LangGraph) observes changes to the blackboard and invites relevant agents to contribute. This creates a dynamic, emergent workflow where the solution is built piece by piece by the most relevant expert at each stage.

### Role in a Large-Scale System: Facilitating Adaptive Collaboration in Dynamic Environments

This architecture is ideal for complex problems where the exact solution path cannot be predefined. It shines in sense-making and analysis tasks:
- **Intelligence Analysis:** Multiple agents (a geo-analyst, a signals analyst, a human-source analyst) all post findings to a shared report.
- **Scientific Discovery:** A system where agents propose experiments, run simulations, and analyze results, posting everything to a shared research log.
- **Complex Debugging:** A code analysis agent, a log parser, and a network traffic analyzer collaborate to find the root cause of a system failure.

We will build a customer support ticket processing system with three specialist agents collaborating on a blackboard. We will demonstrate how this decoupled approach leads to a more **accurate and contextually-rich** final output compared to a single agent.

## Part 1: Setup and Environment

We will need `langchain-community` for its vector stores and embedding models to create our knowledge base, and `tavily-python` for the search tool.

In [ ]:
%pip install -U langchain langgraph langsmith langchain-huggingface transformers accelerate bitsandbytes torch langchain-community sentence-transformers faiss-cpu tavily-python

### 1.2: API Keys and Environment Configuration

We will need our LangSmith and Hugging Face keys.

In [ ]:
import os
import getpass

def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

_set_env("LANGCHAIN_API_KEY")
_set_env("HUGGING_FACE_HUB_TOKEN")

# Configure LangSmith for tracing
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "Industrial - Blackboard Collaboration"

## Part 2: Components for the Blackboard System

This system requires a knowledge base for our retriever, several specialist agents, and a central blackboard state to coordinate them.

### 2.1: The Language Model (LLM)

We will use `meta-llama/Meta-Llama-3-8B-Instruct` as the cognitive engine for all our specialist agents.

In [ ]:
from langchain_huggingface import HuggingFacePipeline
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch

model_id = "meta-llama/Meta-Llama-3-8B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.bfloat16, device_map="auto", load_in_4bit=True)
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=2048, do_sample=False)
llm = HuggingFacePipeline(pipeline=pipe)

print("LLM Initialized. Ready to power our collaborative agents.")

LLM Initialized. Ready to power our collaborative agents.


### 2.2: Creating a Knowledge Base and Retriever Tool

Our Solution Retriever agent needs a knowledge base to search. We will create a simple in-memory vector store using FAISS and Hugging Face embeddings. This will serve as our simulated company help center.

In [ ]:
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.tools import tool

# 1. Create mock knowledge base documents
kb_docs = [
    "Article 1: To reset your Aura Smart Ring, press and hold the small button on the charger while the ring is docked. Hold for 10 seconds until the light flashes white.",
    "Article 2: The QuantumLeap processor supports a maximum of 128GB of DDR5 RAM. Using more than this can cause system instability.",
    "Article 3: Our Smart Mug's battery is designed to last for 2 hours on a full charge when actively heating. It can last up to 8 hours in standby. Charging takes approximately 90 minutes.",
    "Article 4: To resolve app connectivity issues with the Aura Ring, ensure your phone's Bluetooth is enabled, the ring is charged, and you are running the latest version of the Aura app. If problems persist, try restarting your phone."
]

# 2. Create an embedding model
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# 3. Create a FAISS vector store and retriever
vectorstore = FAISS.from_texts(kb_docs, embedding=embeddings)
retriever = vectorstore.as_retriever(search_kwargs={"k": 2})

@tool
def search_knowledge_base(query: str) -> str:
    """Searches the company's knowledge base for solutions to customer problems."""
    print(f"--- [Tool Call] Searching KB for: {query} ---")
    docs = retriever.invoke(query)
    return "\n".join([doc.page_content for doc in docs])

print(f"Knowledge Base created with {len(kb_docs)} documents.")

Knowledge Base created with 4 documents.


### 2.3: Structured Data Models (Pydantic)

These schemas define the objects that our agents will post to the blackboard. They are the language of collaboration for our system.

In [ ]:
from langchain_core.pydantic_v1 import BaseModel, Field
from typing import List, Literal, Optional

class ProblemAnalysis(BaseModel):
    """Structured analysis of the user's problem."""
    product: str = Field(description="The product the user is having an issue with.")
    problem_summary: str = Field(description="A concise, one-sentence summary of the technical problem.")
    user_sentiment: Literal["Positive", "Negative", "Neutral"] = Field(description="The user's sentiment.")

class Solution(BaseModel):
    """A potential solution retrieved from the knowledge base."""
    relevant_articles: List[str] = Field(description="A list of knowledge base articles relevant to the problem.")

class DraftResponse(BaseModel):
    """The final, drafted response to the user."""
    response_text: str = Field(description="The complete, user-facing response drafted by the agent.")

## Part 3: Building the Blackboard Graph

Now we define the state (our blackboard), the specialist agents (nodes), and the central router that orchestrates their collaboration.

### 3.1: Defining the Graph State (The Blackboard)
The state will hold the initial ticket and all the structured data that the agents contribute over time.

In [ ]:
from typing import TypedDict, Annotated

class BlackboardState(TypedDict):
    # Initial input
    ticket: str
    # Data added by agents
    analysis: Optional[ProblemAnalysis]
    solution: Optional[Solution]
    draft: Optional[DraftResponse]
    # Performance log
    performance_log: Annotated[List[str], operator.add]

### 3.2: Defining the Specialist Agent Nodes

Each node is a specialist that reads from and writes to the blackboard (the state).

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
import time

# Agent 1: Problem Analyzer
analyzer_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a Problem Analyzer. Your job is to read a customer support ticket, identify the product, summarize the problem, and gauge the user's sentiment."),
    ("human", "Please analyze the following ticket:\n\n---\n{ticket}\n---")
])
analyzer_chain = analyzer_prompt | llm.with_structured_output(ProblemAnalysis)

def analyzer_node(state: BlackboardState):
    print("--- [AGENT: Problem Analyzer] Activating... ---")
    start_time = time.time()
    result = analyzer_chain.invoke({"ticket": state['ticket']})
    execution_time = time.time() - start_time
    log = f"[Analyzer] Completed in {execution_time:.2f}s."
    print(log)
    return {"analysis": result, "performance_log": [log]}

In [ ]:
from langchain.agents import create_tool_calling_agent, AgentExecutor

# Agent 2: Solution Retriever
retriever_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a Solution Retriever. Use the knowledge base search tool to find articles relevant to the user's problem."),
    ("human", "Problem: {problem_summary}. Please find a solution.")
])
retriever_agent = create_tool_calling_agent(llm, [search_knowledge_base], retriever_prompt)
retriever_executor = AgentExecutor(agent=retriever_agent, tools=[search_knowledge_base])

def retriever_node(state: BlackboardState):
    print("--- [AGENT: Solution Retriever] Activating... ---")
    start_time = time.time()
    response = retriever_executor.invoke({"problem_summary": state['analysis'].problem_summary})
    solution = Solution(relevant_articles=[response['output']])
    execution_time = time.time() - start_time
    log = f"[Retriever] Completed in {execution_time:.2f}s."
    print(log)
    return {"solution": solution, "performance_log": [log]}

In [ ]:
# Agent 3: Draftsman
draftsman_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a Support Response Draftsman. Your job is to write a helpful, empathetic, and clear response to a user based on the problem analysis and retrieved solutions."),
    ("human", "Original Ticket:\n{ticket}\n\nProblem Analysis:\n{analysis}\n\nRetrieved Solution Articles:\n{articles}\n\nPlease draft a response.")
])
draftsman_chain = draftsman_prompt | llm.with_structured_output(DraftResponse)

def draftsman_node(state: BlackboardState):
    print("--- [AGENT: Draftsman] Activating... ---")
    start_time = time.time()
    result = draftsman_chain.invoke({
        "ticket": state['ticket'],
        "analysis": state['analysis'].json(),
        "articles": "\n".join(state['solution'].relevant_articles)
    })
    execution_time = time.time() - start_time
    log = f"[Draftsman] Completed in {execution_time:.2f}s."
    print(log)
    return {"draft": result, "performance_log": [log]}

### 3.3: Defining the Central Router

The router is the controller of our blackboard system. After each step, it inspects the blackboard (the state) and decides which agent should be activated next. This is the core of the event-driven, opportunistic collaboration.

In [ ]:
def router(state: BlackboardState) -> str:
    print("--- [ROUTER] Inspecting blackboard... ---")
    if state.get('draft'):
        print("--- [ROUTER] Decision: Draft is complete. Finishing workflow. ---")
        return END
    if state.get('solution'):
        print("--- [ROUTER] Decision: Solution found. Activating Draftsman. ---")
        return "draftsman"
    if state.get('analysis'):
        print("--- [ROUTER] Decision: Analysis complete. Activating Solution Retriever. ---")
        return "retriever"
    # This should not be reached in this simple graph, but is good practice
    return "analyzer" # Default to analyzer if no other state is present

### 3.4: Assembling the Graph

We connect all nodes to a central conditional edge that uses our router. This creates a star-like topology where the router directs traffic.

In [ ]:
from langgraph.graph import StateGraph, START, END

workflow = StateGraph(BlackboardState)

# Add the nodes
workflow.add_node("analyzer", analyzer_node)
workflow.add_node("retriever", retriever_node)
workflow.add_node("draftsman", draftsman_node)

# The entry point is always the analyzer
workflow.add_edge(START, "analyzer")

# Define the routing logic
workflow.add_conditional_edges(
    "analyzer",
    router,
    {"retriever": "retriever", "draftsman": "draftsman", END: END}
)
workflow.add_conditional_edges(
    "retriever",
    router,
    {"draftsman": "draftsman", END: END}
)
workflow.add_conditional_edges(
    "draftsman",
    router,
    {END: END}
)

app = workflow.compile()

print("Graph constructed and compiled successfully.")
print("The blackboard system is ready for collaboration.")

Graph constructed and compiled successfully.
The blackboard system is ready for collaboration.


## Part 4: Running the Blackboard System

Let's process a sample support ticket and observe how the agents collaborate by posting their findings to the blackboard.

In [ ]:
import json

ticket = "I'm really frustrated. My new Aura Ring isn't syncing my sleep data to the app. I've tried everything! It seems to track my heart rate just fine during the day, but in the morning, there's no sleep info. What do I do?"
inputs = {"ticket": ticket}

step_counter = 1
final_state = None
for output in app.stream(inputs, stream_mode="values"):
    node_name = list(output.keys())[-1] # Get the last updated node
    print(f"\n{'*' * 100}")
    print(f"**Step {step_counter}: {node_name.replace('_', ' ').title()} Node Execution**")
    print(f"{'*' * 100}")
    step_counter += 1
    final_state = output
    print(f"\n{'-' * 100}")
    print("Blackboard State Change:")
    if node_name == "analyzer":
        print("The Analyzer has activated, read the ticket, and posted a structured `analysis` object to the blackboard. The Router sees this new information and decides to activate the Retriever next.")
    elif node_name == "retriever":
        print("The Retriever saw the problem summary, activated, and used its tool to find relevant articles. It has now posted a `solution` object containing these articles to the blackboard. The Router sees this and activates the Draftsman.")
    elif node_name == "draftsman":
        print("The Draftsman activated because it saw both an `analysis` and a `solution` on the blackboard. It synthesized this information into a final response and posted it as a `draft`. The Router sees the completed draft and terminates the workflow.")
    print(f"{'-' * 100}")

****************************************************************************************************
**Step 1: Analyzer Node Execution**
****************************************************************************************************
--- [AGENT: Problem Analyzer] Activating... ---
[Analyzer] Completed in 4.55s.

--- [ROUTER] Inspecting blackboard... ---
--- [ROUTER] Decision: Analysis complete. Activating Solution Retriever. ---

----------------------------------------------------------------------------------------------------
Blackboard State Change: The Analyzer has activated, read the ticket, and posted a structured `analysis` object to the blackboard. The Router sees this new information and decides to activate the Retriever next.
----------------------------------------------------------------------------------------------------

****************************************************************************************************
**Step 2: Retriever Node Execution**
*********